# 1. ロボットイベントデータジェネレーター
<img src="generator.png" alt="generator" width="500"/>

このスクリプトはロボットイベントデータをJSON Lines形式で生成し、ファイルに保存します。

## 主な特徴
- **ロボットID**: `robot_001` ~ `robot_005`
- **コマンドタイプ**: `move_to_point`, `pick_item`, `place_item`, `charge`, `scan_area`
- **サンプル数**: 10,000
- **出力ファイル**: `sample_json_output/robot_data_10000.json`

## イベント構造
- **event_id**: UUID形式の一意なイベントID
- **robot_id**: ロボット識別子
- **timestamp**: ISO8601 UTC時刻
- **location**: x, y, z座標（移動を反映）
- **event**:
  - `event_type`: `"command_execution"`
  - `command`: コマンド名、ターゲット座標、実行時間（ms）、成功ステータス
- **sensors**:
  - バッテリー: 残量（%）、温度（℃）
  - LiDAR: 障害物検知、0/90/180/270°での距離測定

## 動作概要
1. 初期位置（0,0,0）から開始
2. ランダムなコマンドを実行し、位置を更新
3. バッテリー/LiDARセンサー値をランダム生成
4. 各イベントをJSONに変換し、1行ずつ保存
5. イベント間の時間間隔: **5～10秒**

## 結果
- 指定した数（`10,000`）のイベントデータレコードがファイルに保存されます

In [0]:
import json
import random
import uuid
from datetime import datetime, timedelta
import os

# Basic settings
robot_ids = [f"robot_{i:03}" for i in range(1, 6)]
commands = ["move_to_point", "pick_item", "place_item", "charge", "scan_area"]
start_time = datetime(2025, 7, 25, 9, 0, 0)
sample_count = 10_000
output_dir = "sample_json_output"
output_file = "robot_data_10000.json"

# Initial location settings
locations = {r: {"x": 0.0, "y": 0.0, "z": 0.0} for r in robot_ids}
random.seed(42)

os.makedirs(output_dir, exist_ok=True)

# Data generation function
def generate_event(timestamp, robot_id, prev_location):
    command = random.choice(commands)
    duration = random.randint(500, 8000)  # ms
    success = random.random() > 0.1

    # Move location
    delta_x = random.uniform(-5, 5)
    delta_y = random.uniform(-5, 5)
    new_location = {
        "x": round(prev_location["x"] + delta_x, 2),
        "y": round(prev_location["y"] + delta_y, 2),
        "z": 0.0
    }

    return {
        "event_id": str(uuid.uuid4()),
        "robot_id": robot_id,
        "timestamp": timestamp.isoformat() + "Z",
        "location": new_location,
        "event": {
            "event_type": "command_execution",
            "command": {
                "name": command,
                "parameters": {
                    "target_x": round(new_location["x"] + random.uniform(-1, 1), 2),
                    "target_y": round(new_location["y"] + random.uniform(-1, 1), 2)
                },
                "duration_ms": duration,
                "success": success
            }
        },
        "sensors": {
            "battery": {
                "level_percent": round(max(0, 100 - 0.05 * duration + random.uniform(-2, 2)), 1),
                "temperature_c": round(random.uniform(25, 45), 1)
            },
            "lidar": {
                "obstacle_detected": random.random() < 0.2,
                "scan_points": [
                    {"angle": a, "distance": round(random.uniform(0.5, 5.0), 2)}
                    for a in range(0, 360, 90)
                ]
            }
        }
    }, new_location

# Generate and save
with open(os.path.join(output_dir, output_file), "w") as f:
    current_time = start_time
    for i in range(sample_count):
        robot_id = random.choice(robot_ids)
        prev_loc = locations[robot_id]
        record, new_loc = generate_event(current_time, robot_id, prev_loc)
        f.write(json.dumps(record) + "\n")
        locations[robot_id] = new_loc
        # Increase time by 5~10 seconds
        current_time += timedelta(seconds=random.randint(5, 10))

print(f"✅ Generation complete: {sample_count} records → '{output_dir}/{output_file}'")

# 2. オートローダーテスト用 単一JSONジェネレーター

このスクリプトは**Databricks Autoloader**テスト用に個別のJSONファイルを生成します。  
既存のデータディレクトリに新しいファイルを追加し、Autoloaderの新規ファイル検出動作を検証できます。

## 主な特徴
- 単一イベントのJSONファイルを生成（例: `json2.json`, `json3.json`）
- コマンドやセンサー値はランダム生成
- 出力先: `sample_json_output/`

## イベント構造
- **event_id**: UUID形式の一意なイベントID
- **robot_id**: ロボット識別子
- **timestamp**: ISO8601 UTC時刻
- **location**: x, y, z座標
- **event**:
  - コマンド名、ターゲット座標、実行時間（ms）、成功ステータス
- **sensors**:
  - バッテリー残量（%）、温度（℃）
  - LiDAR障害物検知および4方向の距離測定

In [0]:
import json
import random
import uuid
from datetime import datetime
import os

def generate_single_json(filename, timestamp, robot_id):
    commands = ["move_to_point", "pick_item", "place_item", "charge", "scan_area"]
    command = random.choice(commands)
    duration = random.randint(800, 6000)
    success = random.random() > 0.1

    location = {
        "x": round(random.uniform(0, 100), 2),
        "y": round(random.uniform(0, 100), 2),
        "z": 0.0
    }

    record = {
        "event_id": str(uuid.uuid4()),
        "robot_id": robot_id,
        "timestamp": timestamp.isoformat() + "Z",
        "location": location,
        "event": {
            "event_type": "command_execution",
            "command": {
                "name": command,
                "parameters": {
                    "target_x": round(location["x"] + random.uniform(-1, 1), 2),
                    "target_y": round(location["y"] + random.uniform(-1, 1), 2)
                },
                "duration_ms": duration,
                "success": success
            }
        },
        "sensors": {
            "battery": {
                "level_percent": round(random.uniform(50, 95), 1),
                "temperature_c": round(random.uniform(25, 40), 1)
            },
            "lidar": {
                "obstacle_detected": random.random() < 0.2,
                "scan_points": [
                    {"angle": a, "distance": round(random.uniform(0.5, 5.0), 2)}
                    for a in range(0, 360, 90)
                ]
            }
        }
    }

    output_dir = "sample_json_output"
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, filename), "w") as f:
        f.write(json.dumps(record) + "\n")

    print(f"✅ 생성 완료: {filename}")

# 실제 생성 실행
generate_single_json("json2.json", datetime(2025, 7, 26, 10, 0, 0), "robot_007")
generate_single_json("json3.json", datetime(2025, 7, 26, 11, 30, 0), "robot_008")
